# NUTNR

### Purpose
The purpose of this notebook is to calculate the values needed to populate the QARTOD value tables for the Gross Range and Climatology tests as implemented by OOI. The ```NUTNR``` is the sensor name given to the instruments deployed by OOI for measuring seawater nitrate. That includes both the In-Situ Ultraviolet Spectrophotometer (ISUS) and the SUbmersible Ultraviolet Nitrate Analyzer (SUNA) instruments. The ISUS and SUNA instruments were deployed on fixed-depth assets on both coastal and global arrays. The ISUS was phased out in 2018 due to design and data quality issues in favor of the SUNA. Consequently, only the SUNA data is utilized to calculate the QARTOD table inputs.



### ISUS v SUNA
The following is **Table 3.1** from the **"OOI Biogeochemical Sensor Data Best Practices and User Guide" Chapter 3 - Nitrate** which describes which instrument was deployed where and for how long:

| Array | Platforms | Sensors | OOI Class-Series |
| ----- | --------- | ------- | ---------------- |
| Global Argentine Basin Array | Global Profiling Glider | SUNA (2016 - 2017) | NUTNR-M |
|                              | NSIF<br>Subsurface Buoy | ISUS (2015 - 2018) | NUTNR-B |
| Global Irminger Sea Array | GLOBAL Profiling Glider | SUNA (2014 - Present | NUTNR-M |
|                           | NSIF<br>Subsurface Buoy | ISUS (Sep 2014 - Jun 2018)<br>SUNA V2 (Jun 2018 - present) | NUTNR-B |
| Global Southern Ocean Array | Global Profiling Glider | SUNA (2014 - Present) | NUTNR-M |
|                           | NSIF<br>Subsurface Buoy | ISUS (Sep 2014 - Jun 2018)<br>SUNA V2 (Jun 2018 - present) | NUTNR-B |
| Global Station Papa Array | Global Profiling Glider | SUNA V2 (2013 - Present) | NUTNR-M |
| Regional Cabled Array | Shallow Profiler Mooring | Deep SUNA (2014 - Present) | NUTNR-A |
| Coastal Endurance Array | Surface Piercing Profiler | SUNA V2 (2014 - Present) | NUTNR-J |
|                           | NSIF<br>Subsurface Buoy | ISUS (2014 - 2018)<br>SUNA V2 (2018 - present) | NUTNR-B |
| Coastal Pioneer Array | Coastal AUV | SUNA (2015 - Present) | NUTNR-N |
|                       | Surface Piercing Profiler | SUNA V2 (2014 - 2016) | NUTNR-J |
|                       | Coastal Profiling Glider | SUNA (2015 - Present) | NUTNR-M |
|                       | NSIF<br>Subsurface Buoy | ISUS (2013 - Mar 2018)<br>SUNA V2 (Mar 2018 - present) | NUTNR-B |

### Test Parameters

| Dataset Name | OOINet Name | Range |
| ------------ | ----------- | ----- |
| corrected_nitrate_concentrations | salinity_corrected_nitrate | -1.5 - 3000 $\mu$mol/kg |
| nitrate_concentration | nitrate_concentration | -2 - 3000 $\mu$mol/kg |


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

#### Import libraries

In [ ]:
# Import libraries
import os, sys, datetime, pytz, re
import dateutil.parser as parser
import pandas as pd
import numpy as np
import xarray as xr
import warnings
import gc
import json
warnings.filterwarnings("ignore")

In [ ]:
from dask.diagnostics import ProgressBar

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

#### Import the ```ooinet``` M2M toolbox
This toolbox is publicly available at https://github.com/reedan88/OOINet. It should be cloned onto your machine and the setup instructions followed before use.

In [ ]:
# Import the OOINet M2M tool
sys.path.append("/home/areed/Documents/OOI/reedan88/ooinet/")
from ooinet import M2M
from ooinet.utils import convert_time, ntp_seconds_to_datetime, unix_epoch_time
from ooinet.Instrument.common import process_file, add_annotation_qc_flag

#### Import ```ooi_data_explorations``` toolbox
This toolbox is publicly available at https://github.com/oceanobservatories/ooi-data-explorations. Similarly to the ```ooinet``` toolbox above, it should be installed onto your machine following the setup instructions before use.

In [ ]:
sys.path.append("/home/areed/Documents/OOI/oceanobservatories/ooi-data-explorations/python/")
from ooi_data_explorations.common import get_annotations, get_vocabulary, load_gc_thredds
from ooi_data_explorations.combine_data import combine_datasets
from ooi_data_explorations.uncabled.process_nutnr import suna_datalogger, suna_instrument
from ooi_data_explorations.qartod.qc_processing import identify_blocks, create_annotations, process_gross_range, \
    process_climatology, parse_qc, inputs, ANNO_HEADER, CLM_HEADER, GR_HEADER

#### Import plotting and visualization tools

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from ooi_data_explorations.qartod.plotting import *

---
## Identify Data Streams
This section is necessary to identify all of the data stream associated with a specific instrument. This can be done by querying UFrame and iteratively walking through all of the API endpoints. The results are saved into a csv file so this step doesn't have to be repeated each time.

First, set the instrument to search for using OOI terminology:

In [ ]:
instrument = "NUTNR"

### Query OOINet for Data Streams <br>
If the data streams for a given instrument have not yet been identified from OOINet, then want to query OOINet for the data sets and save them to the local memory:

In [ ]:
try:
    datasets = pd.read_csv("../data/NUTNR_datasets.csv")
except:
    datasets = M2M.search_datasets(instrument="NUTNR", English_names=True)
    datasets.to_csv(f"../data/NUTNR_datasets.csv", index=False)
datasets.head()

Separate out the CGSN datasets from the EA and RCA datasets:

In [ ]:
cgsn = datasets["array"].apply(lambda x: True if x.startswith(("CP","GA","GI","GP","GS")) else False)
datasets = datasets[cgsn]

For the Argentine Basin, I will need the Global Profiling Glider SUNA data because we lack any SUNA information from that location

In [ ]:
argentine_moas = datasets["array"].apply(lambda x: True if "GA05MOAS" in x else False)
argentine_gliders = datasets[argentine_moas]
argentine_gliders.head()

Otherwise, remove the ```NUTNRs``` mounted on gliders and AUVs ("MOAS")

In [ ]:
moas = datasets["array"].apply(lambda x: True if "MOAS" in x else False)
datasets = datasets[~moas]
datasets

---
## Single Reference Designator
The reference designator acts as a key for an instrument located at a specific location. First, select a reference designator (refdes) to request data from OOINet.

In [ ]:
reference_designators = sorted(datasets["refdes"])
print("Number of reference designators: " + str(len(reference_designators)))
for refdes in reference_designators:
    print(refdes)

Select a single reference designator

In [ ]:
k=0
refdes = reference_designators[k]
print(refdes)

#### Sensor Vocab
The vocab provides information about the instrument model and type, its location (with descriptive names), depth, and manufacturer. Get the vocab for the given reference designator.

In [ ]:
vocab = M2M.get_vocab(refdes)
vocab

#### Sensor Deployments
Download the deployment information for the selected reference designator:

In [ ]:
deployments = M2M.get_deployments(refdes)
deployments

#### Sensor Data Streams
Next, select the specific data streams for the given reference designator

In [ ]:
datastreams = M2M.get_datastreams(refdes)
datastreams

#### Filter datastreams for just SUNA streams
Midway through the program OOI switched from the ISUS to the SUNA. The ISUS data is generally poor quality and we want to completely exclude it from our analysis.

In [ ]:
mask = datastreams["stream"].apply(lambda x: True if "suna" in x else False)
datastreams[mask]

---
### Argentine Basin
For the Argentine Basin, we do not have any non-ISUS data. Will take a look at the Glider SUNA data as a fill-in for the mooring data

In [ ]:
def preprocess_gliders(ds):
    ds = process_file(ds)
    ds = trim_overlaps(ds, deployments)
    gc.collect()
    return ds

In [ ]:
gliders_data = {}
gliders_annotations = {}

for glider_refdes in argentine_gliders["refdes"]:
    print(f"####### Getting data for {glider_refdes} #######")
    
    # First, get the associated deployments
    glider_deployments = M2M.get_deployments(glider_refdes)
    
    # Next, get the associated metadata
    glider_metadata = M2M.get_metadata(glider_refdes)
    
    # Filter for the relevant science parameters
    science_vars = filter_science_parameters(glider_metadata)
    science_vars = science_vars.groupby(by=["refdes","method","stream"]).agg(lambda x: pd.unique(x.values.ravel()).tolist())
    science_vars = science_vars.reset_index()
    science_vars = science_vars.applymap(lambda x: x[0] if len(x) == 1 else x)
    
    # Get the datastreams
    glider_datastreams = M2M.get_datastreams(glider_refdes)
    
    # Get the associated annotations
    refdes_annotations = M2M.get_annotations(glider_refdes)
    gliders_annotations.update({glider_refdes: refdes_annotations})

    # Get the available datasets
    for index in glider_datastreams.index:
        # Get the method and stream
        method = glider_datastreams.loc[index]["method"]
        stream = glider_datastreams.loc[index]["stream"]

        # Get the URL - first try the goldCopy thredds server
        thredds_url = M2M.get_thredds_url(glider_refdes, method, stream, goldCopy=True)

        # Get the catalog
        catalog = M2M.get_thredds_catalog(thredds_url)

        # Clean the catalog
        catalog = M2M.clean_catalog(catalog, stream, deployments)

        # Get the links to the THREDDs server and load the data
        dodsC = M2M.URLS["goldCopy_dodsC"]
        if method == "telemetered":
            tele_files = [re.sub("catalog.html\?dataset=", dodsC, file) for file in catalog]
            print(f"----- Load {method}-{stream} data -----")
            with ProgressBar():
                tele_data = xr.open_mfdataset(tele_files, preprocess=preprocess_gliders, parallel=True)
        elif method == "recovered_host":
            host_files = [re.sub("catalog.html\?dataset=", dodsC, file) for file in catalog]
            print(f"----- Load {method}-{stream} data -----")
            with ProgressBar():
                host_data = xr.open_mfdataset(host_files, preprocess=preprocess_gliders, parallel=True)
        elif method == "recovered_inst":
            inst_files = [re.sub("catalog.html\?dataset=", dodsC, file) for file in catalog]
            print(f"----- Load {method}-{stream} data -----")
            with ProgressBar():
                inst_data = xr.open_mfdataset(inst_files, preprocess=preprocess_gliders, parallel=True)
        else:
            pass
        
                
    # Merge all the datasets together
    methods = glider_datastreams["method"].unique()
    if "telemetered" not in methods:
        tele_data = None
    if "recovered_host" not in methods:
        host_data = None
    if "recovered_inst" not in methods:
        inst_data = None
    refdes_data = combine_datasets(tele_data, host_data, inst_data, None)
    
    # Add in the annotations
    refdes_data = add_annotation_qc_flag(refdes_data, refdes_annotations)
    
    # Save the results
    gliders_data.update({glider_refdes: refdes_data})

Combine all of the glider datasets together into a single dataset


In [ ]:
merged_gliders = None
for refdes in gliders_data.keys():
    if merged_gliders is None:
        merged_gliders = gliders_data[refdes]
    else:
        merged_gliders = xr.concat([merged_gliders, gliders_data[refdes]], dim="time")

Sort the merged data based on time


In [ ]:
merged_gliders = merged_gliders.sortby("time")
merged_gliders

---
---
## Metadata 
The metadata contains the following important key pieces of data for each reference designator: **method**, **stream**, **particleKey**, and **count**. The method and stream are necessary for identifying and loading the relevant dataset. The particleKey tells us which data variables in the dataset we should be calculating the QARTOD parameters for. The count lets us know which dataset (the recovered instrument, recovered host, or telemetered) contains the most data and likely has the best record to use to calculate the QARTOD tables. 

In [ ]:
metadata = M2M.get_metadata(refdes)
metadata

#### Sensor Parameters
Each instrument returns multiple parameters containing a variety of low-level instrument output and metadata. However, we are interested in science-relevant parameters for calculating the relevant QARTOD test limits. We can identify the science parameters based on the preload database, which designates the science parameters with a "data level" of L1 or L2. 

Consequently, we through several steps to identify the relevant parameters. First, we query the preload database with the relevant metadata for a reference designator. Then, we filter the metadata for the science-relevant data streams. 

In [ ]:
def filter_science_parameters(metadata):
    """This function returns the science parameters for each datastream"""
    
    def filter_parameter_ids(pdId, pid_dict):
        data_level = pid_dict.get(pdId)
        if data_level is not None:
            if data_level > 0:
                return True
            else:
                return False
        else:
            return False
    
    # Filter the parameters for processed science parameters
    data_levels = M2M.get_parameter_data_levels(metadata)
    mask = metadata["pdId"].apply(lambda x: filter_parameter_ids(x, data_levels))
    metadata = metadata[mask]

    return metadata

def filter_metadata(metadata):
    science_vars = filter_science_parameters(metadata)
    # Next, eliminate the optode temperature from the stream
    mask = science_vars["particleKey"].apply(lambda x: False if "temp" in x else True)
    science_vars = science_vars[mask]
    science_vars = science_vars.groupby(by=["refdes","method","stream"]).agg(lambda x: pd.unique(x.values.ravel()).tolist())
    science_vars = science_vars.reset_index()
    science_vars = science_vars.applymap(lambda x: x[0] if len(x) == 1 else x)
    science_vars = science_vars.explode(column="particleKey")
    return science_vars

In [ ]:
science_vars = filter_science_parameters(metadata)
science_vars = science_vars.groupby(by=["refdes","method","stream"]).agg(lambda x: pd.unique(x.values.ravel()).tolist())
science_vars = science_vars.reset_index()
science_vars = science_vars.applymap(lambda x: x[0] if len(x) == 1 else x)
science_vars

In [ ]:
science_vars

---
## Load Data
When calculating the QARTOD data tables, we only want to utilize the most complete data record available for a given reference designator. We do this by getting all the available data streams, loading the data, and then combining them into a single dataset. 

In [ ]:
def trim_overlaps(ds, deployments):
    """Trim overlapping deployment data (necessary to use xr.open_mfdataset)"""
    # --------------------------------
    # Second, get the deployment times
    deployments = deployments.sort_values(by="deploymentNumber")
    deployments = deployments.set_index(keys="deploymentNumber")
    # Shift the start times by (-1) 
    deployEnd = deployments["deployStart"].shift(-1)
    # Find where the deployEnd times are earlier than the deployStart times
    mask = deployments["deployEnd"] > deployEnd
    # Wherever the deployEnd times occur after the shifted deployStart times, replace those deployEnd times
    deployments["deployEnd"][mask] = deployEnd[mask]
    deployments["deployEnd"] = deployments["deployEnd"].apply(lambda x: pd.to_datetime(x))
    
    # ---------------------------------
    # With the deployments info, can write a preprocess function to filter 
    # the data based on the deployment number
    depNum = np.unique(ds["deployment"])
    deployInfo = deployments.loc[depNum]
    deployStart = deployInfo["deployStart"].values[0]
    deployEnd = deployInfo["deployEnd"].values[0]
    
    # Select the dataset data which falls within the specified time range
    ds = ds.sel(time=slice(deployStart, deployEnd))
    
    return ds

def preprocess_datalogger(ds):
    ds = process_file(ds)
    ds = suna_datalogger(ds)
    ds = trim_overlaps(ds, deployments)
    gc.collect()
    return ds

def preprocess_instrument(ds):
    ds = process_file(ds)
    ds = suna_instrument(ds)
    ds = trim_overlaps(ds, deployments)
    gc.collect()
    return ds

---
## Download Data
To access data, there are two applicable methods. The first is to download the data and save the netCDF files locally. The second is to access and process the files remotely on the THREDDS server, without having to download the data.

In [ ]:
# Get the available datasets
for index in datastreams[mask].index:
    # Get the method and stream
    method = datastreams.loc[index]["method"]
    stream = datastreams.loc[index]["stream"]

    # Get the URL - first try the goldCopy thredds server
    thredds_url = M2M.get_thredds_url(refdes, method, stream, goldCopy=True)

    # Get the catalog
    catalog = M2M.get_thredds_catalog(thredds_url)

    # Clean the catalog
    catalog = M2M.clean_catalog(catalog, stream, deployments)
    
    # Get the links to the THREDDs server and load the data
    dodsC = M2M.URLS["goldCopy_dodsC"]
    if method == "telemetered":
        tele_files = [re.sub("catalog.html\?dataset=", dodsC, file) for file in catalog]
        print(f"----- Load {method}-{stream} data -----")
        with ProgressBar():
            tele_data = xr.open_mfdataset(tele_files, preprocess=preprocess_datalogger, parallel=True)
    elif method == "recovered_host":
        host_files = [re.sub("catalog.html\?dataset=", dodsC, file) for file in catalog]
        print(f"----- Load {method}-{stream} data -----")
        with ProgressBar():
            host_data = xr.open_mfdataset(host_files, preprocess=preprocess_datalogger, parallel=True)
    elif method == "recovered_inst":
        inst_files = [re.sub("catalog.html\?dataset=", dodsC, file) for file in catalog]
        print(f"----- Load {method}-{stream} data -----")
        with ProgressBar():
            inst_data = xr.open_mfdataset(inst_files, preprocess=preprocess_instrument, parallel=True)
    else:
        pass

**Combine the data into a single file**

In [ ]:
data = combine_datasets(tele_data, host_data, inst_data, None)
data

**Clean up workspace variables and free up memory**

In [ ]:
host_data.close()
tele_data.close()
inst_data.close()
del tele_data, host_data, inst_data
gc.collect()

---
## Human-in-the-loop review
Next, visualize and inspect the available time series and data streams that are to be tested in order to identify instrument failures or data issues that are known failure modes.

#### Annotations
The annotations associated with a specific reference designator may contain relevant information on the performance or reliability of the data for a given dataset. The annotations are downloaded from OOINet as a json and processed into a pandas dataframe. Each annotation may apply to the entire dataset, to a specific stream, or to a specific variable. With the downloaed annotations, we can use the information contained in the ```qcFlag``` column to translate the annotations into QC flags, which can then be used to filter out bad data. 

In [ ]:
def human_dates(x):
    if pd.isna(x):
        x = datetime.datetime.now()
    else:
        x = convert_time(x)
    return x

In [ ]:
annotations = M2M.get_annotations(refdes)

# convert the times to human-readable-dates
annotations_with_dates = annotations
annotations_with_dates["beginDT"] = annotations_with_dates["beginDT"].apply(lambda x: human_dates(x))
annotations_with_dates["endDT"] = annotations_with_dates["endDT"].apply(lambda x: human_dates(x))
annotations_with_dates

Look at specific annotations to see what they say:

In [ ]:
annotations_with_dates.loc[4]["annotation"]

Add the annotations to the dataset:

In [ ]:
annotations = M2M.get_annotations(refdes)

In [ ]:
data = add_annotation_qc_flag(data, annotations)
data

**Plot some of the variables/data for visual inspection of the time series**

In [ ]:
fig, ax = plot_variable(data, 'corrected_nitrate_concentration')

ax.plot(data.where(data.rollup_annotations_qc_results > 2, drop=True)["time"], data.where(data.rollup_annotations_qc_results > 2, drop=True)['corrected_nitrate_concentration'],
        marker='o', linestyle="", color="yellow")

In [ ]:
fig, ax = plot_variable(data, 'nitrate_concentration')

ax.plot(data.where(data.rollup_annotations_qc_results > 2, drop=True)["time"], data.where(data.rollup_annotations_qc_results > 2, drop=True)['nitrate_concentration'],
        marker='o', linestyle="", color="yellow")

In [ ]:
fig, ax = plot_variable(data, "sea_water_practical_salinity")
ax.plot(data.where(data.rollup_annotations_qc_results > 2, drop=True)["time"], data.where(data.rollup_annotations_qc_results > 2, drop=True)['sea_water_practical_salinity'],
        marker='o', linestyle="", color="yellow")

Mask out identified bad data

In [ ]:
# # CP03ISSM-RID26-07-NUTNRB000
#mask = (data.time >= pd.to_datetime("2018-07-25 12:00:00")) & (data.time <= pd.to_datetime("2018-10-26 19:52:00"))
#data["rollup_annotations_qc_results"][mask] = 4

#mask = (data.time >= pd.to_datetime("2022-08-17 00:00:00")) & (data.time <= pd.to_datetime("2022-11-13 13:55:00"))
#data["rollup_annotations_qc_results"][mask] = 4

# # CP04OSSM-RID26-07-NUTNRB000
#mask = (data.time >= pd.to_datetime("2020-07-25")) & (data.time <= pd.to_datetime("2020-11-08 13:13:00"))
#data["rollup_annotations_qc_results"][mask] = 4

# # GI01SUMO-RID16-07-NUTNRB000
#mask = (data.time >= pd.to_datetime("2019-02-07")) & (data.time <= pd.to_datetime("2019-08-09 08:04:00"))
#data["rollup_annotations_qc_results"][mask] = 4

#mask = (data.sea_water_practical_salinity < 34.0)
#data["rollup_annotations_qc_results"][mask] = 4

# # GI01SUMO-SBD11-08-NUTNRB000
#mask = (data.corrected_nitrate_concentration > 40)
#data["rollup_annotations_qc_results"][mask] = 4

Select just the "good" data

In [ ]:
good_data = data.where(data.rollup_annotations_qc_results <= 3, drop=True)

Lastly, clean out any duplicate time stamps

In [ ]:
_, index = np.unique(good_data['time'], return_index=True)
good_data = good_data.isel(time=index)

---
## Gross Range
The Gross Range QARTOD test consists of two parameters: a fail range which indicates when the data is bad, and a suspect range which indicates when data is either questionable or interesting. The fail range values are set based upon the instrument/measurement and associated calibration. For example, the conductivity sensors are calibration for measurements between 0 (freshwater) and 9 (highly-saline waters). The suspect range values are calculated based on the mean of the available data $\pm$3$\sigma$.

In [ ]:
from ooi_data_explorations.qartod.gross_range import GrossRange
from ooi_data_explorations.qartod.plotting import *
from ooi_data_explorations.qartod.qc_processing import format_gross_range, format_climatology

#### Test Parameters & Sensor Ranges
Map out the data variables in the data set to the data stream inputs and the associated sensor ranges

In [ ]:
test_parameters = {
    "nitrate_concentration": [-2, 3000],
    "corrected_nitrate_concentration": [-1.5, 3000],
}

**Argentine Basin**: Need to subset the glider data to only get the observations that are at comparable depths to the mooring instruments

In [ ]:
# Subset the glider_data
good_data = merged_gliders.where((merged_gliders.depth < 5) & (merged_gliders.depth > 0), drop=True)
good_data["nitrate_concentration"] = good_data.sci_suna_nitrate_um
good_data["corrected_nitrate_concentration"] = good_data.sci_suna_nitrate_um

**Calculate the Gross Range Values**

In [ ]:
site, node, sensor = refdes.split("-", 2)
gross_range_table = pd.DataFrame()


for param in test_parameters:
    sensor_range = test_parameters.get(param)
    if param == "corrected_nitrate_concentration":
        inp = "salinity_corrected_nitrate"
    else:
        inp = param
    
    if param in good_data.variables:
        print(f"##### Calculating gross range for {param} #####")
        # Check if there is enough data
        if len(good_data[param].dropna(dim="time")) < 100:
            user_range = [np.nan, np.nan]
            source = "Not enough data to calculate user range."
        else:
            gross_range = GrossRange(sensor_range[0], sensor_range[1])
            gross_range.fit(good_data, param, check_normality=True)
            user_range = [gross_range.suspect_min, gross_range.suspect_max]
            source = gross_range.source
        # Check which streams have the param in it
        streams = metadata[metadata["particleKey"] == inp]["stream"].unique()
        for stream in streams:
            qc_dict = format_gross_range(inp, sensor_range, user_range, site, node, sensor, stream, source)
            gross_range_table = gross_range_table.append(qc_dict, ignore_index=True)
            
        # Plot the result
        try:
            fig, ax = plot_gross_range(good_data, param, gross_range)
            ymin, ymax = ax.get_ylim()
            if ymin < sensor_range[0]:
                ymin = sensor_range[0]
            if ymax > sensor_range[1]:
                ymax = sensor_range[1]
            ax.set_ylim((ymin, ymax))
            #ax.set_ylim((200, 600))
        except:
            pass

**Add the stream name and the source comments**

In [ ]:
gross_range_table['notes'] = ('User range based on data collected through {}.'.format("2021-01-01"))
gross_range_table

**Check the results**

In [ ]:
for ind in gross_range_table.index:
    print(gross_range_table.loc[ind]["qcConfig"])

**Save the gross range table**

In [ ]:
gross_range_table.to_csv(f"../results/gross_range/{refdes}.csv", index=False, columns=GR_HEADER)

---
## Climatology
For the climatology QARTOD test, First, we bin the data by month and take the mean. The binned-montly means are then fit with a 2 cycle harmonic via Ordinary-Least-Squares (OLS) regression. Ranges are calculated based on the 3$\sigma$ calculated from the OLS-fitting.  

In [ ]:
from ooi_data_explorations.qartod.climatology import Climatology

In [ ]:
def make_climatology_table(ds, param, tinp, zinp, sensor_range, depth_bins):
    """Function which calculates the climatology table based on the """
    
    climatologyTable = pd.DataFrame()
    
    if depth_bins is None:
        # Filter out the data outside the sensor range
        m = (ds[param] > sensor_range[0]) & (ds[param] < sensor_range[1]) & (~np.isnan(ds[param]))
        param_data = ds[param][m]
        
        # Fit the climatology for the selected data
        pmin, pmax = [0, 0]
        
        try:
            climatology = Climatology()
            climatology.fit(param_data)

            # Create the depth index
            zspan = pd.interval_range(start=pmin, end=pmax, periods=1, closed="both")

            # Create the monthly bins
            tspan = pd.interval_range(0, 12, closed="both")

            # Calculate the climatology data
            vmin = climatology.monthly_fit - climatology.monthly_std*3
            vmin = np.floor(vmin*10000)/10000
            for vind in vmin.index:
                if vmin[vind] < sensor_range[0] or vmin[vind] > sensor_range[1]:
                    vmin[vind] = sensor_range[0]
            vmax = climatology.monthly_fit + climatology.monthly_std*3
            for vind in vmax.index:
                if vmax[vind] < sensor_range[0] or vmax[vind] > sensor_range[1]:
                    vmax[vind] = sensor_range[1]
            vmax = np.floor(vmax*10000)/10000
            vdata = pd.Series(data=zip(vmin, vmax), index=vmin.index).apply(lambda x: [v for v in x])
            vspan = vdata.values.reshape(1,-1)

            # Build the climatology dataframe
            climatologyTable = climatologyTable.append(pd.DataFrame(data=vspan, columns=tspan, index=zspan))

        except:
            # Here is where to create nans if insufficient data to fit
            # Create the depth index
            zspan = pd.interval_range(start=pmin, end=pmax, periods=1, closed="both")

            # Create the monthly bins
            tspan = pd.interval_range(0, 12, closed="both")

            # Create a series filled with nans
            vals = []
            for i in np.arange(len(tspan)):
                vals.append([np.nan, np.nan])
            vspan = pd.Series(data=vals, index=tspan).values.reshape(1,-1)

            # Add to the data
            climatologyTable = climatologyTable.append(pd.DataFrame(data=vspan, columns=tspan, index=zspan))
            
        del ds, vspan, tspan, zspan
        gc.collect()
        
    else:        
    # Iterate through the depth bins to calculate the climatology for each depth bin
        for dbin in depth_bins:
            # Get the pressure range to bin from
            pmin, pmax = dbin[0], dbin[1]

            # Select the data from the pressure range
            bin_data = data.where((data[zinp] >= pmin) & (data[zinp] <= pmax), drop=True)

            # sort based on time and make sure we have a monotonic dataset
            bin_data = bin_data.sortby('time')
            _, index = np.unique(bin_data['time'], return_index=True)
            bin_data = bin_data.isel(time=index)

            # Filter out the data outside the sensor range
            m = (bin_data[param] > sensor_range[0]) & (bin_data[param] < sensor_range[1]) & (~np.isnan(bin_data[param]))
            param_data = bin_data[param][m]

            # Fit the climatology for the selected data
            try:
                climatology = Climatology()
                climatology.fit(param_data)

                # Create the depth index
                zspan = pd.interval_range(start=pmin, end=pmax, periods=1, closed="both")

                # Create the monthly bins
                tspan = pd.interval_range(0, 12, closed="both")

                # Calculate the climatology data
                vmin = climatology.monthly_fit - climatology.monthly_std*3
                vmin = np.floor(vmin*10000)/10000
                for vind in vmin.index:
                    if vmin[vind] < sensor_range[0] or vmin[vind] > sensor_range[1]:
                        vmin[vind] = sensor_range[0]
                vmax = climatology.monthly_fit + climatology.monthly_std*3
                vmax = np.floor(vmax*10000)/10000
                for vind in vmax.index:
                    if vmax[vind] < sensor_range[0] or vmax[vind] > sensor_range[1]:
                        vmax[vind] = sensor_range[1]
                vdata = pd.Series(data=zip(vmin, vmax), index=vmin.index).apply(lambda x: [v for v in x])
                vspan = vdata.values.reshape(1,-1)

                # Build the climatology dataframe
                climatologyTable = climatologyTable.append(pd.DataFrame(data=vspan, columns=tspan, index=zspan))

            except:
                # Here is where to create nans if insufficient data to fit
                # Create the depth index
                zspan = pd.interval_range(start=pmin, end=pmax, periods=1, closed="both")

                # Create the monthly bins
                tspan = pd.interval_range(0, 12, closed="both")

                # Create a series filled with nans
                vals = []
                for i in np.arange(len(tspan)):
                    vals.append([np.nan, np.nan])
                vspan = pd.Series(data=vals, index=tspan).values.reshape(1,-1)

                # Add to the data
                climatologyTable = climatologyTable.append(pd.DataFrame(data=vspan, columns=tspan, index=zspan))

            del climatology, bin_data, vspan, tspan, zspan
            gc.collect()
    
    return climatologyTable, climatology

**Get the depth bins and filter based on max depth.** <br>
For the ```SUNAs``` which are only deployed on Surface Moorings, there is no depth bins needed.

In [ ]:
depth_bins = None

In [ ]:
# Initialize the climatology lookup table
climatologyLookup = pd.DataFrame()

# Setup the Table Header
TBL_HEADER = ["[1,1]","[2,2]","[3,3]","[4,4]","[5,5]","[6,6]","[7,7]","[8,8]","[9,9]","[10,10]","[11,11]","[12,12]"]

# Set the subsite-node-sensor
subsite, node, sensor = refdes.split("-", 2)

# Iterate through the parameters
for param in test_parameters:
    if param in good_data.variables:
        if param == "corrected_nitrate_concentration":
            inp = "salinity_corrected_nitrate"
        else:
            inp = param
        
        # ----------------- Depth tables ---------------------
        # Get the sensor range of the parameter to test
        print(f"##### Calculating climatology for {param} #####")
        sensor_range = test_parameters.get(param)
        # Set the sensor range to something more reasonable, i.e. 
        if param == 'nitrate_concentration':
            sensor_range = (-2.0, 3000)
        else:
            sensor_range = (-1.5, 3000)
        
        # Generate the climatology table with the depth bins
        climatologyTable, climatology = make_climatology_table(good_data, param, "time", "depth", sensor_range, depth_bins)
        
        # Get the variance
        try:
            variance = float(np.round(climatology.regression['variance_explained']*100, 1))
        except:
            variance = 0.0

        # Create the tableName
        tableName = f"{refdes}-{param}.csv"
        
        # Save the results
        climatologyTable.to_csv(f"../results/climatology/climatology_tables/{tableName}", header=TBL_HEADER)
        
        # ------------------ Lookup tables ------------------
        # Check which streams have the param in it
        streams = metadata[metadata["particleKey"] == inp]["stream"].unique()
        for stream in streams:
            if stream == "metbk_hourly":
                pass
            else:
                qc_dict = {
                    "subsite": subsite,
                    "node": node,
                    "sensor": sensor,
                    "stream": stream,
                    "parameters": {
                        "inp": inp,
                        "tinp": "time",
                        "zinp": "depth",
                    },
                    "climatologyTable": f"climatology_tables/{refdes}-{param}.csv",
                    "source": f"The variance explained by the climatology model is {variance}%.",
                    "notes": "Climatology based on available data through 2021-01-01."
                }
                # Append to the lookup table
                climatologyLookup = climatologyLookup.append(qc_dict, ignore_index=True)
            
        # ------------------ Plot the climatology ------------------
        if good_data[param].time.size > 100000:
            try:
                subset = sorted(np.random.choice(good_data.time, 100000, replace=False))
                subset_data = good_data.sel(time=subset)
                fig, ax = plot_climatology(subset_data, param, climatology)
                #ax.set_ylim((sensor_range[0], sensor_range[1]))
                del subset, subset_data
                gc.collect()
            except:
                pass
        else:
            try:
                fig, ax = plot_climatology(good_data, param, climatology)
                #ax.set_ylim((sensor_range[0], sensor_range[1]))
            except:
                pass

**Check the last climatologyTable for reasonableness**

In [ ]:
climatologyTable

**Check the climatologyLookup table that all the entries made it in**

In [ ]:
climatologyLookup

**Save the climatologyLookup table**

In [ ]:
# Save the lookup table results
climatologyLookup.to_csv(f"../results/climatology/{refdes}.csv", index=False, columns=CLM_HEADER)